In [1]:
import torch.nn as nn
import torch
from contextlib import nullcontext
import time
from tqdm.notebook import tqdm
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 环境与设备

In [2]:
import torch

print(f"CUDA is available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB")

CUDA is available: True
Number of GPUs: 1
  GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
    Memory: 8.00 GB


# 参数配置

In [10]:
from model.model_minimind import MiniMindConfig
class TrainArgs():
    #文件管理
    out_dir = './sft_output'
    checkpoint_path = "./sft_output/latest_checkpoint.pth"
    data_path = '../data/sft_1024.jsonl'
    #神经网络训练管理
    epochs = 2
    batch_size = 16
    accumulation_steps = 4
    learning_rate = 5e-4
    warm_up = 0
    grad_clip = 1
    dtype = 'bfloat16'
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    num_workers = 0
    log_interval = 50
    save_interval = 2000
    ctx = nullcontext() if device == "cpu" else torch.cuda.amp.autocast()
class LLMargs():
    use_moe = True
    hidden_size = 512
    num_hidden_layers = 8
    

lm_config = MiniMindConfig(use_moe=LLMargs.use_moe,hidden_size=LLMargs.hidden_size,num_hidden_layers=LLMargs.num_hidden_layers)


# 加载模型与数据

加载模型

In [5]:
from transformers import AutoTokenizer
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
tokenizer = AutoTokenizer.from_pretrained('../model/')
model = MiniMindForCausalLM(lm_config).to(TrainArgs.device)



In [7]:


ckp = './pretrain_output/latest_checkpoint.pth'
state_dict = torch.load(ckp, map_location=TrainArgs.device)
model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['model.embed_tokens.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.mlp.experts.0.gate_proj.weight', 'model.layers.0.mlp.experts.0.down_proj.weight', 'model.layers.0.mlp.experts.0.up_proj.weight', 'model.layers.0.mlp.experts.1.gate_proj.weight', 'model.layers.0.mlp.experts.1.down_proj.weight', 'model.layers.0.mlp.experts.1.up_proj.weight', 'model.layers.0.mlp.experts.2.gate_proj.weight', 'model.layers.0.mlp.experts.2.down_proj.weight', 'model.layers.0.mlp.experts.2.up_proj.weight', 'model.layers.0.mlp.experts.3.gate_proj.weight', 'model.layers.0.mlp.experts.3.down_proj.weight', 'model.layers.0.mlp.experts.3.up_proj.weight', 'model.layers.0.mlp.gate.weight', 'model.layers.0.mlp.shared_experts.0.gate_proj.weight', 'model.lay

In [8]:
print(model)

total_params = sum(p.numel() for p in model.parameters())

# 打印结果，使用 f-string 的格式化功能让数字更易读（例如，加上千位分隔符）
print(f"模型总参数量 (Total Parameters): {total_params:,}")

# 如果你还想区分“可训练参数”，可以这样做：
# 在大多数情况下，总参数量和可训练参数量是一样的。
# 除非你手动设置了某些参数的 requires_grad=False (冻结了某些层)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"可训练参数量 (Trainable Parameters): {trainable_params:,}")

MiniMindForCausalLM(
  (model): MiniMindModel(
    (embed_tokens): Embedding(6400, 512)
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-7): 8 x MiniMindBlock(
        (self_attn): Attention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=128, bias=False)
          (v_proj): Linear(in_features=512, out_features=128, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
        (mlp): MOEFeedForward(
          (experts): ModuleList(
            (0-3): 4 x FeedForward(
              (gate_proj): Linear(in_features=512, out_features=1408, bias=False)
              (down_proj): Linear(in_features=1408, out_features=512, bias=False)
           

加载数据

In [15]:
from torch.utils.data import Dataset, DataLoader, Subset
import json
class SFTDataset(Dataset):
    def __init__(self,  tokenizer,data_path, max_length=1024):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(data_path)
        self.bos_id = tokenizer('<|im_start|>assistant', add_special_tokens=False).input_ids
        self.eos_id = tokenizer('<|im_end|>', add_special_tokens=False).input_ids

    def __len__(self):
        return len(self.samples)

    def load_data(self, data_path):
        samples = []
        with open(data_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())
                samples.append(data)
        return samples

    def _create_chat_prompt(self, conversations):
        """构建符合ChatML格式的对话"""
        messages = []
        for i, turn in enumerate(conversations):
            role = 'user' if i % 2 == 0 else 'assistant'
            messages.append({"role": role, "content": turn['content']})
        return self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

    def _generate_loss_mask(self, input_ids):
        loss_mask = [0] * len(input_ids)
        i = 0
        while i < len(input_ids):
            if input_ids[i:i + len(self.bos_id)] == self.bos_id:
                start = i + len(self.bos_id)
                end = start
                while end < len(input_ids):
                    if input_ids[end:end + len(self.eos_id)] == self.eos_id:
                        break
                    end += 1
                for j in range(start + 1, min(end + len(self.eos_id) + 1, self.max_length)):
                    loss_mask[j] = 1
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids)
            else:
                i += 1
        return loss_mask

    def __getitem__(self, index):
        sample = self.samples[index]
        # 构建对话提示
        prompt = self._create_chat_prompt(sample['conversations'])
        input_ids = self.tokenizer(prompt).input_ids[:self.max_length]
        input_ids += [self.tokenizer.pad_token_id] * (self.max_length - len(input_ids))

        # 生成动态损失掩码
        loss_mask = self._generate_loss_mask(input_ids)

        # 构建训练数据
        X = input_ids[:-1].clone().detach()
        Y = input_ids[1:].clone().detach()
        loss_mask = loss_mask[1:].clone().detach() # 对齐预测位置

        return X, Y, loss_mask

In [1]:
train_ds = SFTDataset(tokenizer,TrainArgs.data_path,max_length=512)

NameError: name 'SFTDataset' is not defined

In [11]:
print(f'{train_ds[1]}\n数据集总长度为{len(train_ds)}')

NameError: name 'train_ds' is not defined

In [ ]:
test_ds = Subset(train_ds,range(1000))

加载dataloader

In [ ]:
test_loader = DataLoader(
    test_ds,
    batch_size = TrainArgs.batch_size,
    shuffle = True,
    num_workers = TrainArgs.num_workers,
    pin_memory= True
)

In [ ]:
train_loader = DataLoader(
    train_ds,
    batch_size = TrainArgs.batch_size,
    shuffle = True,
    num_workers = TrainArgs.num_workers,
    pin_memory= True
)

检查一下

In [ ]:
tag = False
for batch_idx, (X_batch, Y_batch, loss_mask_batch) in enumerate(train_loader):
    if not tag:
        print(f"Batch {batch_idx}:")
        print(f"  X_batch shape: {X_batch.shape}")
        print(f"  Y_batch shape: {Y_batch.shape}")
        print(f"  loss_mask_batch shape: {loss_mask_batch.shape}")
        tag = True
    break

In [ ]:
tag = False
for batch_idx, (X_batch, Y_batch, loss_mask_batch) in enumerate(test_loader):
    if not tag:
        print(f"Batch {batch_idx}:")
        print(f"  X_batch shape: {X_batch.shape}")
        print(f"  Y_batch shape: {Y_batch.shape}")
        print(f"  loss_mask_batch shape: {loss_mask_batch.shape}")
        tag = True
    break

# 训练

In [ ]:
import math
from torch import optim
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))
optimizer = optim.AdamW(model.parameters(), lr=TrainArgs.learning_rate)
#据模型在训练数据上的表现来调整模型的参数  训练的一个大脑
scaler = torch.cuda.amp.GradScaler(enabled=(TrainArgs.dtype in ['float16', 'bfloat16']))
#自动混合精度训练工具 训练的一个助手

In [ ]:
import swanlab
# 初始化swanlab，传入项目名、实验名等
swanlab.init(
    project="MiniMind-SFT", 
    experiment_name="MiniMind-SFT", 
    config=vars(TrainArgs()) # 将你的所有超参数配置一次性传给swanlab
)

In [ ]:
def train_epoch(epoch, model, train_loader, optimizer, scaler, lm_config):
    #选择初始化损失函数！
    model.train() # 确保模型处于训练模式

    loss_fct = nn.CrossEntropyLoss(reduction='none')

    start_time = time.time()#一会记录日志要用

    iter_per_epoch = len(train_loader)#看一下一次数据有多长，计算学习率要用

    # --- 新增：使用tqdm包装DataLoader以显示进度条 ---
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{TrainArgs.epochs}", leave=True)

    for step, (X, Y, loss_mask) in enumerate(train_loader):#不断提取数据
        X = X.to(TrainArgs.device)
        Y = Y.to(TrainArgs.device)
        loss_mask = loss_mask.to(TrainArgs.device)#数据上设备，上到gpu上

        
        lr = get_lr(epoch * iter_per_epoch + step, TrainArgs.epochs * iter_per_epoch, TrainArgs.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr                #计算当前学习率的算法

        with TrainArgs.ctx:
            res = model(X)#得到的是logits，aux_loss

            loss = loss_fct(
                res.logits.view(-1, res.logits.size(-1)),
                Y.view(-1)
            ).view(Y.size())  #计算loss的算法
            loss = (loss * loss_mask).sum() / loss_mask.sum()
            loss += res.aux_loss
            loss = loss / TrainArgs.accumulation_steps  

            scaler.scale(loss).backward()  #反向传播

        if (step + 1) % TrainArgs.accumulation_steps == 0:#判断梯度累加到位没有
            scaler.unscale_(optimizer)#*前置操作
            torch.nn.utils.clip_grad_norm_(model.parameters(), TrainArgs.grad_clip)
            #梯度裁剪
            scaler.step(optimizer)#参数更新
            scaler.update()#*调整

            optimizer.zero_grad(set_to_none=True)#本次训练结束，清零梯度，为下一次反向传播准备

        if step % TrainArgs.log_interval == 0:
            spend_time = time.time() - start_time
            current_lr = optimizer.param_groups[0]['lr']
            progress_bar.set_postfix({
                "loss": f"{loss.item():.3f}",
                "lr": f"{current_lr:.2e}"
            })

            if (swanlab is not None) : #用日志软件记录训练过程
                swanlab.log({"loss": loss.item() * TrainArgs.accumulation_steps,
                           "lr": optimizer.param_groups[-1]['lr'],
                           "epoch_Time": spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60})

            # 保存checkpoint
        if (step + 1) % TrainArgs.save_interval == 0:
            # 只在更新步之后保存，确保梯度累积完成
            if (step + 1) % TrainArgs.accumulation_steps == 0:
                model.eval()
                print(f"\nSaving checkpoint at step {step+1}...")
                
                # 创建checkpoint字典
                checkpoint = {
                     # 对模型权重进行半精度转换
                    'model': {k: v.half() for k, v in model.state_dict().items()},
                    # 对优化器状态也进行转换（更严谨的做法）
                    'optimizer': {
                        'state': {k: v.half() if torch.is_tensor(v) else v for k, v in optimizer.state_dict()['state'].items()},
                        'param_groups': optimizer.state_dict()['param_groups']
                    },
                    'scaler': scaler.state_dict(),
                    'epoch': epoch,
                    'step': step,
                    'config': TrainArgs,
                }
                
                # 保存
                torch.save(checkpoint, TrainArgs.checkpoint_path)
                print("Checkpoint saved.")
                model.train() # 切换回训练模式


# 实现循环训练

In [ ]:
# --- 关键：Checkpoint加载逻辑 ---
start_epoch = 0
start_step = 0
if os.path.exists(TrainArgs.checkpoint_path):
    print(f"=> Resuming from checkpoint: {TrainArgs.checkpoint_path}")
    checkpoint = torch.load(TrainArgs.checkpoint_path, map_location=TrainArgs.device)
    
    # 加载模型权重 (处理DDP保存的权重)
    model_state_dict = checkpoint['model']
    if any(key.startswith('module.') for key in model_state_dict):
            model_state_dict = {k.replace('module.', ''): v for k, v in model_state_dict.items()}
    model.load_state_dict(model_state_dict)
    
    # 加载优化器和scaler状态
    optimizer.load_state_dict(checkpoint['optimizer'])
    scaler.load_state_dict(checkpoint['scaler'])
    
    # 恢复训练进度
    start_epoch = checkpoint['epoch']
    # 注意：恢复step逻辑可以让dataloader跳过已训练数据，这里简化为从头开始当前epoch
    print(f"=> Resumed from epoch {start_epoch + 1}")
else:
    print("=> Starting from scratch...")

model.to(TrainArgs.device)

In [ ]:
#测试训练运行以下代码
for epoch in range(start_epoch, TrainArgs.epochs):
        train_epoch(epoch, model, test_loader, optimizer, scaler, lm_config)

In [ ]:
#正式训练运行以下代码
for epoch in range(start_epoch, TrainArgs.epochs):
        train_epoch(epoch, model, train_loader, optimizer, scaler, lm_config)